# Aumento de Datos

El aumento de datos se realizó mediante transformaciones aleatorias aplicadas con albumentations, incluyendo volteos, rotaciones, escalados y ajustes de brillo y contraste. Las coordenadas de los aneurismas se transformaron de forma consistente junto con la imagen, y solo se conservaron aquellas muestras que mantuvieron la patología visible y las dimensiones originales. Las nuevas imágenes generadas se integraron con los datos originales para el entrenamiento del modelo.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [6]:
data_completo=pd.read_csv('dataset_completo_npy.csv')
data_completo

,SeriesInstanceUID,SOPInstanceUID,npy_path,x,y,label
0,1.2.826.0.1.3680043.8.498.10004044428023505108...,1.2.826.0.1.3680043.8.498.66911171872877028453...,1.2.826.0.1.3680043.8.498.66911171872877028453...,-1.000000,-1.000000,0
1,1.2.826.0.1.3680043.8.498.10004684224894397679...,1.2.826.0.1.3680043.8.498.10587118170829166010...,1.2.826.0.1.3680043.8.498.10587118170829166010...,-1.000000,-1.000000,0
2,1.2.826.0.1.3680043.8.498.10014757658335054766...,1.2.826.0.1.3680043.8.498.76601744454891248086...,1.2.826.0.1.3680043.8.498.76601744454891248086...,-1.000000,-1.000000,0
3,1.2.826.0.1.3680043.8.498.10021411248005513321...,1.2.826.0.1.3680043.8.498.81593405716626656866...,1.2.826.0.1.3680043.8.498.81593405716626656866...,-1.000000,-1.000000,0
4,1.2.826.0.1.3680043.8.498.10040419508532196461...,1.2.826.0.1.3680043.8.498.11401266297286362999...,1.2.826.0.1.3680043.8.498.11401266297286362999...,-1.000000,-1.000000,0
...,...,...,...,...,...,...
2642,1.2.826.0.1.3680043.8.498.99887675554378211308...,1.2.826.0.1.3680043.8.498.10885430363476327277...,1.2.826.0.1.3680043.8.498.10885430363476327277...,204.232497,216.433289,1
2643,1.2.826.0.1.3680043.8.498.99887675554378211308...,1.2.826.0.1.3680043.8.498.75978746530527925899...,1.2.826.0.1.3680043.8.498.75978746530527925899...,153.421602,188.455472,1
2644,1.2.826.0.1.3680043.8.498.99892390884723813599...,1.2.826.0.1.3680043.8.498.88559536225248716051...,1.2.826.0.1.3680043.8.498.88559536225248716051...,364.014870,195.330855,1
2645,1.2.826.0.1.3680043.8.498.99892390884723813599...,1.2.826.0.1.3680043.8.498.41770479778255354615...,1.2.826.0.1.3680043.8.498.41770479778255354615...,177.486989,210.557621,1


In [ ]:
import albumentations as A
import cv2
import os
from tqdm import tqdm

# Se definen los parámetros globales para el proceso de aumento.
# N_GENERAR indica cuántas versiones nuevas se crearán por cada imagen original de aneurisma.
N_GENERAR = 1  
input_folder = '/home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/'
output_csv_name = 'dataset_dataaugmentation.csv'

# Se filtra el dataset completo para obtener únicamente las filas correspondientes a casos positivos (aneurismas).
# Esto se hace porque el objetivo es balancear las clases aumentando solo los ejemplos de la clase minoritaria.
df_aneurismas = data_completo[data_completo['label'] == 1].reset_index(drop=True)

# Se define la tubería (pipeline) de transformaciones utilizando la librería Albumentations.
# Se configura 'remove_invisible=True' para descartar automáticamente los puntos clave (aneurismas) 
# que queden fuera de la imagen tras una transformación geométrica.
transform = A.Compose([
    # Se aplica un volteo horizontal con una probabilidad del 50%.
    A.HorizontalFlip(p=0.5),
    
    # Se aplica un volteo vertical con una probabilidad del 50%.
    A.VerticalFlip(p=0.5),
    
    # Se aplican transformaciones afines aleatorias (desplazamiento, escalado y rotación).
    # border_mode=cv2.BORDER_REFLECT rellena los bordes vacíos reflejando la imagen original.
    A.ShiftScaleRotate(
        shift_limit=0.05,  # Límite de desplazamiento
        scale_limit=0.1,   # Límite de zoom
        rotate_limit=10,   # Límite de rotación en grados
        border_mode=cv2.BORDER_REFLECT,
        p=0.7              # Probabilidad de aplicación del 70%
    ),
    
    # Se altera aleatoriamente el brillo y el contraste para simular diferentes condiciones de adquisición.
    A.RandomBrightnessContrast(p=0.2),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=True))

nuevas_filas = []

# Se itera sobre cada registro del DataFrame de aneurismas mostrando una barra de progreso.
for idx, row in tqdm(df_aneurismas.iterrows(), total=len(df_aneurismas)):
    
    # Se construye la ruta completa.
    path = os.path.join(input_folder, row['npy_path'])
    
    if not os.path.exists(path):
        print(f"Advertencia: Archivo no encontrado, saltando: {path}")
        continue
        
    image = np.load(path)
    
    # Se definen las coordenadas del aneurisma como un punto clave (keypoint) para que se transforme junto con la imagen.
    keypoints = [(row['x'], row['y'])]

    # Se genera el número especificado de imágenes aumentadas por cada original.
    for i in range(N_GENERAR):
        try:
            # Se aplica la transformación a la imagen y a sus puntos clave.
            transformed = transform(image=image, keypoints=keypoints)
            aug_img = transformed['image'].astype(np.float32)
            aug_kps = transformed['keypoints']

            # Se verifica si el punto clave del aneurisma sigue presente en la imagen transformada.
            # Si la lista 'aug_kps' está vacía, significa que el punto se salió de los límites y la imagen se descarta.
            if len(aug_kps) == 0:
                continue
            
            # Se extraen las nuevas coordenadas transformadas.
            new_x, new_y = aug_kps[0]

            # Se valida que la imagen resultante mantenga las dimensiones esperadas (512x512x3).
            if aug_img.shape != (512, 512, 3):
                print(f"Saltando por tamaño incorrecto: {aug_img.shape}")
                continue
            
            # Se realiza una validación de seguridad adicional para asegurar que las coordenadas estén dentro del rango de la imagen.
            if not (0 <= new_x < 512 and 0 <= new_y < 512):
                continue

            # Se genera un nuevo nombre de archivo único para la imagen aumentada.
            original_name = row['npy_path'].replace('.npy', '')
            new_filename = f"{original_name}_aug_{i}.npy"
            
            # Se guarda la nueva imagen transformada en el disco.
            np.save(os.path.join(input_folder, new_filename), aug_img)

            # Se crea una nueva fila con los metadatos actualizados (nueva ruta y nuevas coordenadas).
            new_row = row.copy()
            new_row['npy_path'] = new_filename
            new_row['x'] = new_x
            new_row['y'] = new_y
            
            nuevas_filas.append(new_row)

        except Exception as e:
            # Se capturan y reportan errores individuales sin detener todo el proceso.
            print(f"Error procesando {row['npy_path']}: {e}")

# Se crea un DataFrame con las nuevas filas generadas.
df_nuevos = pd.DataFrame(nuevas_filas)

# Se concatena el dataset original con las nuevas imágenes aumentadas.
df_final = pd.concat([data_completo, df_nuevos], ignore_index=True)

# Se guarda el dataset final extendido en un archivo CSV.
df_final.to_csv(output_csv_name, index=False)

# Se imprimen estadísticas finales sobre la cantidad de imágenes generadas.
print(f"Imágenes nuevas: {len(df_nuevos)}")
print(f"Dataset total: {len(df_final)}")

  2%|▏         | 37/1498 [00:00<00:07, 185.95it/s]

Advertencia: Archivo no encontrado, saltando: /home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/1.2.826.0.1.3680043.8.498.12524552726742591936607820382077304797.npy


  6%|▋         | 96/1498 [00:00<00:07, 188.68it/s]

Advertencia: Archivo no encontrado, saltando: /home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/1.2.826.0.1.3680043.8.498.37968147245973562230977823049232474212.npy
Advertencia: Archivo no encontrado, saltando: /home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/1.2.826.0.1.3680043.8.498.12112808457186454422752724768665392380.npy
Advertencia: Archivo no encontrado, saltando: /home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/1.2.826.0.1.3680043.8.498.97121094041712829608625148100059963709.npy
Advertencia: Archivo no encontrado, saltando: /home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/1.2.826.0.1.3680043.8.498.97121094041712829608625148100059963709.npy


  8%|▊         | 116/1498 [00:00<00:07, 191.04it/s]

...

Advertencia: Archivo no encontrado, saltando: /home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/1.2.826.0.1.3680043.8.498.73337788135479702179192460540805444241.npy


 94%|█████████▎| 1403/1498 [00:07<00:00, 147.38it/s]

Advertencia: Archivo no encontrado, saltando: /home/dayana-henao/Escritorio/Deteccion-de-aneurismas-intracraneales/datos_procesados_npy/1.2.826.0.1.3680043.8.498.61578595799953025665982634553057422851.npy


100%|██████████| 1498/1498 [00:08<00:00, 176.92it/s]


Imágenes nuevas: 1473
Dataset total: 4120
